<a id='Save and Load model'></a> 

# Index
[1.Save and Load model](#Save and Load model) 

[2.Image preprocess pipeline](#Image preprocess pipeline)

[3.Pre-training/Fine-Tuning](#Image preprocess pipeline)

## Save and Load model

<a id='Save and Load model'></a>

save model architecture and weights separatly

In [ ]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

Save/Load the Entire Model

In [ ]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

# Returns a compiled model identical to the previous one
model = load_model('my_model.h5')

## Image preprocess pipeline

<a id='Image preprocess pipeline'></a>

In [ ]:
# use albumentations for better performance
from albumentations import VerticalFlip,HorizontalFlip,ShiftScaleRotate,Resize,Compose

In [ ]:
def create_transform():
    fun = Compose([HorizontalFlip(p=0.25),ShiftScaleRotate(0.1,0.1,45,p=1)])
    def transform(x):
        return fun(image=x)['image']
    return transform

transform = create_transform()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,preprocessing_function=transform)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary')

In [ ]:
# autoAugment
from autoaugment.augmentation_transforms import ImageNetPolicy

In [ ]:
transform_fun = ImageNetPolicy()
train_datagen = ImageDataGenerator(preprocessing_function=transform_fun)

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=100,
                              validation_data=val_generator,
                              max_queue_size=100, workers=4, use_multiprocessing=True)

## Pre-training/Fine-Tuning

<a id='Pre-training/Fine-Tuning'></a>

Fast feature extraction without data augmentation

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size=(150, 150),
                                            batch_size=batch_size,
                                            class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
    if i * batch_size >= sample_count:
        break
    return features, labels

Fast feature extraction with data augmentation

In [ ]:
model = models.Sequential()
conv_base.trainable = False # freeze weights in VGG. DO THIS BEFORE COMPILE or RE-COMPILE!
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
# model.trainable_weights == 4

Fine-tune last block of conv in VGG. Should check model.trainable_weights to ensure right weights are being trained

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers:
    if layer.name in ['block5_conv1','block5_conv2','block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False